In [ ]:
import sys
import numpy as np
import wfdb
import wfdb.processing
import tensorflow as tf
import pandas as pd

import os
import glob
import zipfile
import shutil


from matplotlib import pyplot as plt
from IPython.display import display
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('../Research/Ex Vivo CSV Files LD.zip', 'r')
zip_ref.extractall('../Research/Raw Data/Ex Vivo CSV Files LD/')
zip_ref.close()

In [ ]:
"""
Uploading data and renaming columns
Parameters: 
-----------
path: string
    path to the folder with dataframes 

Returns: 
-------
data_list
    List with pd.DataFrames 
"""

def upload_data(path):

    data_list = []
    files_list = os.path.join(path, '*.csv')
    for filename in glob.glob(files_list):
        data = pd.read_csv(filename, header=0)
        data.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
        data.rename(columns=lambda x: x.replace('_O', '_opt'), inplace=True)
        data.rename(columns=lambda x: x.replace('.', '_opt_'), inplace=True)
        data.rename(columns=lambda x: x.replace('Bsk', filename + '_Bsk'), inplace=True)
        data.rename(columns=lambda x: x.replace(path, ''), inplace=True)
        data.rename(columns=lambda x: x.replace('.csv_annot.csv', ''), inplace=True)
        data.rename(columns=lambda x: x.replace('\\', ''), inplace=True)
        print(filename, data.shape)
        data_list.append(data)
    return(data_list)
path = r'C:\Users\ecath\Desktop\Research\Raw Data\LD dataset'
data_list = upload_data(path)

In [ ]:
def class_splitting(data_list):
    
    drivers = []
    nondrivers = []
    noises = []

    for df in data_list:
        driver = pd.DataFrame()
        nondriver = pd.DataFrame()
        noise = pd.DataFrame()
        for col in df.columns[1::3]:
            curr_index = list(df.columns).index(col)
            prev_index = curr_index - 1
            next_index = curr_index + 1
            prev_col = df.iloc[:,prev_index]
            next_col = df.iloc[:,next_index]
            if df[col][0] == 1:
                driver[df.columns[prev_index]] = prev_col
                driver[df.columns[next_index]] = next_col
            elif df[col][0] == 0:
                nondriver[df.columns[prev_index]] = prev_col
                nondriver[df.columns[next_index]] = next_col
            else:
                noise[df.columns[prev_index]] = prev_col
                noise[df.columns[next_index]] = next_col
        drivers.append(driver)
        nondrivers.append(nondriver)
        noises.append(noise)
        
    return(drivers, nondrivers, noises)

drivers, nondrivers, noises = class_splitting(data_list)

In [ ]:
def del_nul_and_nan(df):
    
    df.fillna(value=0, axis=1, inplace=True)
    
################# add to drop off zero columns ######################    
#     for col in df.columns:
#         if df[col].sum() == 0:
#             df.drop([col], axis = 1, inplace = True)  
################# add to drop off zero columns ######################    
    
    return df

In [ ]:
for i in drivers:
    del_nul_and_nan(i)
for i in nondrivers:
    del_nul_and_nan(i)
for i in noises:
    del_nul_and_nan(i)

In [ ]:
def saving_single_signal(data_list, path, mapping):
    
    if not os.path.exists(path):
        os.mkdir(path)
    os.chdir(path)
    
    for df in data_list:
        dfs = np.split(df, df.shape[1], axis=1)
        
        if mapping == 'el':
            i = 0 
            while i <= len(dfs)-1:
                name = str(df.columns[i]) 
                dfs[i].to_csv(name + '.csv', index=False)
                i+=2
        if mapping == 'opt':
            i = 1 
            while i <= len(dfs):
                name = str(df.columns[i]) 
                dfs[i].to_csv(name + '.csv', index=False)
                i+=2

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers electrode signals LD'
saving_single_signal(drivers, path, 'el')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers electrode signals LD'
saving_single_signal(nondrivers, path, 'el')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noise electrode signals LD'
saving_single_signal(noises, path, 'el')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers optical signals LD'
saving_single_signal(drivers, path, 'opt')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers optical signals LD'
saving_single_signal(nondrivers, path, 'opt')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noise optical signals LD'
saving_single_signal(noises, path, 'opt')

In [ ]:
def saving_wfdb(path, freq):

    records = os.listdir(path)
    records = [os.path.splitext(x)[0] for x in records]

#     print(records)


    if not os.path.exists(path + ' WFDB'):
        os.mkdir(path + ' WFDB')
    os.chdir(path)


# 2. Write the signals and headers at WFDB format
    freq = freq
    for p in records:
        if p != '.ipynb_checkpoints':
            csv_filename = p + '.csv'
            wfdb_filename = p 
            command = 'wrsamp -F ' + freq + ' -i ' + csv_filename + ' -o ' + wfdb_filename + ' -x 200'
            print(command)
            os.system(command)

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers optical signals LD'
saving_wfdb(path, freq='1000')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers optical signals LD'
saving_wfdb(path, freq='1000')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noises optical signals LD'
saving_wfdb(path, freq='1000')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers electrode signals LD'
saving_wfdb(path, freq='1017.25')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers electrode signals LD'
saving_wfdb(path, freq='1017.25')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noises electrode signals LD'
saving_wfdb(path, freq='1017.25')

In [ ]:
def moving_wfdb(source):

#     if not os.path.exists('/home/kate/Research/Raw Data/All optical WFDB'):
#         os.mkdir('/home/kate/Research/Raw Data/All optical WFDB')
#     destination = '/home/kate/Research/Raw Data/All optical WFDB'
    
    destination = source + ' WFDB'

    files = os.listdir(source)

    for file in files:
        if file.endswith('.dat'):
            print(file)
            shutil.move(os.path.join(source, file), os.path.join(destination, file))
        if file.endswith('.hea'):
            print(file)
            shutil.move(os.path.join(source, file), os.path.join(destination, file))

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted drivers electrode signals LD'
moving_wfdb(source)

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted nondrivers electrode signals LD'
moving_wfdb(source)

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted noises electrode signals LD'
moving_wfdb(source)

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted drivers optical signals LD'
moving_wfdb(source)

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted nondrivers optical signals LD'
moving_wfdb(source)

In [ ]:
source = '/home/kate/Research/Raw Data/Splitted noises optical signals LD'
moving_wfdb(source)

In [ ]:
def reference_creation(path, label):
    files = os.listdir(path)
    files = [os.path.splitext(x)[0] for x in files]
#     print(files)

    df = pd.DataFrame(files)
    label = np.full_like(df, fill_value=label)
    label = pd.DataFrame(label)
    df = pd.concat([df, label], axis=1, ignore_index=True)
    return df

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers electrode signals LD'
dr = reference_creation(path, 'D')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers electrode signals LD'
nondr = reference_creation(path, 'ND')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noises electrode signals'
n = reference_creation(path, 'N')

In [ ]:
ref = pd.concat([dr, nondr, n], axis=0, ignore_index=True)
ref.to_csv('/home/kate/Research/Raw Data/REFERENCE electrode LD.csv', index=False, header=False)

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted drivers optical signals LD'
dr_opt = reference_creation(path, 'D')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted nondrivers optical signals LD'
nondr_opt = reference_creation(path, 'D')

In [ ]:
path = '/home/kate/Research/Raw Data/Splitted noises optical signals LD'
n_opt = reference_creation(path, 'D')

In [ ]:
ref_opt = pd.concat([dr_opt, nondr_opt, n_opt], axis=0, ignore_index=True)
ref_opt.to_csv('/home/kate/Research/Raw Data/REFERENCE optical LD.csv', index=False, header=False)

In [ ]:
ref_el = pd.read_csv('/home/kate/Research/Raw Data/REFERENCE electrode LD.csv', header=-1)
ref_opt = pd.read_csv('/home/kate/Research/Raw Data/REFERENCE optical LD.csv', header=-1)
ref_full = pd.concat([ref_el, ref_opt], axis=0, ignore_index=True)
ref_full.to_csv('/home/kate/Research/Raw Data/REFERENCE full LD.csv', index=False, header=False)

In [ ]:
# I moved all obtained wfdb files and REFERENCE file in one folder 

# if not os.path.exists('/home/kate/Research/Raw Data/All WFDB electrode LD'):
#     os.mkdir('/home/kate/Research/Raw Data/All WFDB electrode LD')
  
shutil.copy('/home/kate/Research/Raw Data/REFERENCE full LD.csv', '/home/kate/Research/Raw Data/All WFDB full LD/')

source = '/home/kate/Research/Raw Data/Splitted nondrivers optical signals LD WFDB/'
destination = '/home/kate/Research/Raw Data/All WFDB full LD/'

files = os.listdir(source)

for file in files:
    print(file)
    shutil.copy(os.path.join(source, file), os.path.join(destination, file))

In [ ]:
os.listdir('/home/kate/Research/Raw Data/All WFDB full LD')

In [ ]:
path = '/home/kate/Research/Raw Data/'
os.chdir(path)

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

if __name__ == '__main__':
    zipf = zipfile.ZipFile('Electrode signals WFDB.zip', 'w', zipfile.ZIP_STORED)
    zipdir('/home/kate/Research/Raw Data/All WFDB/', zipf)
    zipf.close()